In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
cols = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']

In [3]:

from sklearn.linear_model.logistic import LogisticRegression
train_lr = train.copy(deep=True)
test_lr = test.copy(deep=True)

train_lr['Attrition'] = train_lr['Attrition'].map(lambda x: 1 if x=='Yes' else 0)
test_lr['Attrition'] = -1
test_lr = test_lr[train_lr.columns]
data = pd.concat([train_lr, test_lr])

data = data.drop(['EmployeeNumber', 'EmployeeCount', 'StandardHours'], axis=1)

for attr in cols:
    new_attr = pd.DataFrame()
    new_attr = pd.get_dummies(data[attr],prefix=attr)
    data = pd.concat([data, new_attr], axis=1)
    data = data.drop([attr], axis=1)
train_lr = data[data['Attrition'] != -1]
test_lr = data[data['Attrition'] == -1]
test_lr = test_lr.drop('Attrition', axis=1)

# 归一化数值比较大的向量
Min_max_cols=['TotalWorkingYears', 'Age', 'MonthlyIncome', 'DailyRate', 'DistanceFromHome', 'HourlyRate']
for col in Min_max_cols:
    min_max = MinMaxScaler()
    train_lr[col] = min_max.fit_transform(train_lr[col].values.reshape(-1, 1))
    test_lr[col] = min_max.transform(test_lr[col].values.reshape(-1, 1))

# 采用LR模型
model_lr = LogisticRegression(max_iter=100, verbose=True, random_state=33, tol=1e-4)
model_lr.fit(train_lr.drop(['user_id', 'Attrition'], axis=1), train_lr['Attrition'])

predict_lr = model_lr.predict_proba(test_lr.drop('user_id', axis=1))[:, 1]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to 

In [4]:

from sklearn.model_selection import train_test_split
import catboost as cb
# catboost的分类
train_cb = train.copy(deep=True)
test_cb = test.copy(deep=True)

train_cb = train_cb.drop(['user_id', 'EmployeeCount', 'EmployeeNumber', 'StandardHours'], axis=1)
test_cb = test_cb.drop(['EmployeeCount', 'EmployeeNumber', 'StandardHours'], axis=1)
train_cb['Attrition'] = train_cb['Attrition'].map(lambda x:1 if x=='Yes' else 0)

x_train, x_valid, y_train, y_valid = train_test_split(train_cb.drop('Attrition', axis=1), train_cb['Attrition'], test_size=0.2, random_state=40)
train_cb.columns
model_cb = cb.CatBoostClassifier(iterations=1000,
                                         depth=7,
                                         learning_rate=0.01,
                                         loss_function='Logloss',
                                         eval_metric='AUC',
                                         logging_level='Verbose',
                                         metric_period=50)



# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(x_train.columns)):
    if x_train.columns.values[i] in cols:
        categorical_features_indices.append(i)
print(categorical_features_indices)

model_cb.fit(x_train, y_train, eval_set=(x_valid, y_valid), cat_features=categorical_features_indices)

predict_cb = model_cb.predict_proba(test_cb.drop('user_id', axis=1))[:, 1]


[1, 3, 6, 8, 12, 14, 18, 19]
0:	test: 0.5807292	best: 0.5807292 (0)	total: 67ms	remaining: 1m 6s
50:	test: 0.8094363	best: 0.8094363 (50)	total: 761ms	remaining: 14.2s
100:	test: 0.8187806	best: 0.8187806 (100)	total: 1.49s	remaining: 13.3s
150:	test: 0.8123468	best: 0.8187806 (100)	total: 2.61s	remaining: 14.7s
200:	test: 0.8121936	best: 0.8187806 (100)	total: 3.42s	remaining: 13.6s
250:	test: 0.8117341	best: 0.8187806 (100)	total: 4.13s	remaining: 12.3s
300:	test: 0.8129596	best: 0.8187806 (100)	total: 4.81s	remaining: 11.2s
350:	test: 0.8158701	best: 0.8187806 (100)	total: 5.49s	remaining: 10.2s
400:	test: 0.8140319	best: 0.8187806 (100)	total: 6.17s	remaining: 9.22s
450:	test: 0.8157169	best: 0.8187806 (100)	total: 6.83s	remaining: 8.31s
500:	test: 0.8143382	best: 0.8187806 (100)	total: 7.52s	remaining: 7.49s
550:	test: 0.8152574	best: 0.8187806 (100)	total: 8.21s	remaining: 6.69s
600:	test: 0.8151042	best: 0.8187806 (100)	total: 8.87s	remaining: 5.89s
650:	test: 0.8167892	best: 0.

In [5]:
# GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier

train_gb = train.copy(deep=True)
test_gb = test.copy(deep=True)

train_gb = train_gb.drop(['user_id', 'EmployeeCount', 'EmployeeNumber', 'StandardHours'], axis=1)
test_gb = test_gb.drop(['EmployeeCount', 'EmployeeNumber', 'StandardHours'], axis=1)
train_gb['Attrition'] = train_gb['Attrition'].map(lambda x:1 if x=='Yes' else 0)

from sklearn.preprocessing import LabelEncoder
lbe_list = []
for col in cols:
    lbe = LabelEncoder()
    train_gb[col] = lbe.fit_transform(train_gb[col])
    test_gb[col] = lbe.transform(test_gb[col])
    lbe_list.append(lbe)   


model_gb = GradientBoostingClassifier(random_state=10)
model_gb.fit(train_gb.drop('Attrition', axis=1), train_gb['Attrition'])
predict_gb = model_gb.predict_proba(test_gb.drop('user_id', axis=1))[:, 1]

In [6]:
predict = ( predict_lr + predict_cb + predict_gb)/3

submission = pd.DataFrame({'user_id': test['user_id'], 'Attrition': predict})
submission.to_csv("submission_test.csv", index=False, sep=',', columns=['user_id', 'Attrition'])

In [15]:
train_gb['Attrition'].value_counts()

0    988
1    188
Name: Attrition, dtype: int64

In [4]:
train.columns

Index(['user_id', 'Age', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate',
       'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'],
      dtype='object')

In [17]:
test_gb.columns

Index(['user_id', 'Age', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField',
       'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus',
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18',
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
       'Attrition'],
      dtype='object')